In [1]:
import pandas as pd
import numpy as np
import hashlib
import sqlite3
import datetime as dt

### Fussball DataVault

In [75]:
def data_vault_hash(string_to_hash: str) -> str:
    """function takes a string, converts every character to uppercase, replaces blankspaces and calculates MD5 of the result"""
    try:
        cleaned_string = string_to_hash.upper().replace(" ", "").encode("utf8")
        md5hash = hashlib.md5(cleaned_string).hexdigest()
        return md5hash
    except:
        return None

In [3]:
def get_column_names(table: str, db) -> list:
    """returns a list of the column names of a table from a SQL database
    if index was created properly"""
    df_column = pd.read_sql(f'PRAGMA table_info({table});', con=db)
    return df_column['name'].to_list()

In [4]:
def get_table_names(db) -> list:
    """returns a list of all the names of the tables of a SQL database"""
    query = (
            'SELECT name FROM sqlite_master '
            'WHERE type IN ("table", "view") '
            'AND name NOT LIKE "sqlite_%"'
            ';'
            )
    table = pd.read_sql(query, con=db)
    return table['name'].to_list()

In [5]:
def get_primary_key(table: str, db) -> str:
    """returns the name of the primary key of a table from a SQL database if index was created properly"""
    try:
        table_idx = pd.read_sql(f'PRAGMA index_list({table});', con=db)
        primary_key_var = (table_idx.loc[table_idx['origin']=='pk']['name'][0])
        primary_key = pd.read_sql(f'PRAGMA index_info({primary_key_var});', con=db)
        return list(primary_key['name'])
    except IndexError:
        return f"Table 'index_info' not found, can't get primary key of {table}"

In [6]:
def get_primary_key2(table: str, db) -> str:
    """returns the name of the primary key of a table from a SQL database if index was created properly"""
    try:
        table_idx = pd.read_sql(f'PRAGMA table_info({table});', con=db)
        primary_key_var = (table_idx.loc[table_idx['pk']== 1]['name'][0])
        #primary_key = pd.read_sql(f'PRAGMA table_info({primary_key_var});', con=db)
        return primary_key_var
    except IndexError:
        return f"Table 'index_info' not found, can't get primary key of {table}"

In [7]:
def get_foreign_key(table:str, db) -> list: 
    keylist = pd.read_sql(f"PRAGMA foreign_key_list({table})", con=db)
    return list(keylist['from'])

In [8]:
db = sqlite3.connect("C:/Users/aliyi/Desktop/ALFATRAINING/dataengineer/abschluss_projekt/Projekt/database.sqlite")
cursor = db.cursor()
# attachDatabaseSQL = "ATTACH DATABASE ':memory:' AS memdb"
# cursor.execute(attachDatabaseSQL)

https://pythontic.com/database/sqlite/attach

**Database is a path-like object giving the pathname (absolute or relative to the current working directory) of the database file to be opened. You can use ":memory:" to open a database connection to a database that resides in RAM instead of on disk.**

**When a database is accessed by multiple connections, and one of the processes modifies the database, the SQLite database is locked until that transaction is committed. The timeout parameter specifies how long the connection should wait for the lock to go away until raising an exception. The default for the timeout parameter is 5.0 (five seconds).**

In [9]:
get_primary_key2("Team", db)

'id'

In [10]:
get_column_names("Team", db)[1]

'team_api_id'

In [11]:
get_primary_key2("Match", db)

'id'

In [12]:
get_foreign_key("Match", db)

['away_player_11',
 'away_player_10',
 'away_player_9',
 'away_player_8',
 'away_player_7',
 'away_player_6',
 'away_player_5',
 'away_player_4',
 'away_player_3',
 'away_player_2',
 'away_player_1',
 'home_player_11',
 'home_player_10',
 'home_player_9',
 'home_player_8',
 'home_player_7',
 'home_player_6',
 'home_player_5',
 'home_player_4',
 'home_player_3',
 'home_player_2',
 'home_player_1',
 'away_team_api_id',
 'home_team_api_id',
 'league_id',
 'country_id']

#### HUB_TABELLEN

In [13]:
def hub_table(source_table: str, db):
    """drops if exist, then creates and inserts a HUB table and save it into database"""
    # primary_key = get_primary_key2(source_table, db)  # get_primary_key function
    primary_key = get_column_names(source_table, db)[1]
    # drops
    sql_drop = f"""DROP TABLE IF EXISTS HUB_{source_table};"""
    cursor.execute(sql_drop)

    # creates
    sql_create = f"""CREATE TABLE HUB_{source_table}(HK TEXT NOT NULL PRIMARY KEY, BK INTEGER, LDTS DATETIME,RS INTEGER,
                FOREIGN KEY(BK) REFERENCES {source_table}({primary_key}));"""
    cursor.execute(sql_create)

    # inserts
    sql_insert = f"""INSERT INTO HUB_{source_table} (HK, BK, LDTS, RS)
                SELECT {primary_key}||'_'||CAST(1 AS STR), {primary_key}, datetime('now'), 1
                FROM {source_table}"""
    cursor.execute(sql_insert)

    # converts DF
    df = pd.read_sql(f'Select * From HUB_{source_table};', con=db)

    # generate hash
    df['HK'] = df['HK'].apply(lambda w: data_vault_hash(w))  # data_vault_hash function

    # load df as a table and foreign keys definieren
    df.to_sql(f'HUB_{source_table}', con=db, if_exists='replace',  index=False,
            dtype={"HK": "TEXT NOT NULL PRIMARY KEY", "BK":"INTEGER", "LDTS": "DATETIME", "RS": "INTEGER"})

    cursor.execute(f"""CREATE TABLE new_HUB_{source_table}(HK TEXT NOT NULL PRIMARY KEY, BK INTEGER, LDTS DATETIME, RS INTEGER,
                                     FOREIGN KEY (BK) REFERENCES {source_table}({primary_key}));""")

    cursor.execute(f"INSERT INTO new_HUB_{source_table} SELECT * FROM HUB_{source_table};")

    cursor.execute(f"DROP TABLE HUB_{source_table};")
    cursor.execute(f"ALTER TABLE new_HUB_{source_table} RENAME TO HUB_{source_table};")

    # dtype={"HK": "TEXT NOT NULL PRIMARY KEY", "BK":"INTEGER", "LDTS": "DATETIME", "RS": "INTEGER", "FOREIGN KEY(BK)" : f"REFERENCES {source_table}({primary_key})"})
    # df_HUB = pd.read_sql(f'Select * From {hub_table};', con=db)
    # return df_HUB

In [40]:
source_list_HUB = ["player", "league", "team"]
for table in source_list_HUB:
    hub_table(table, db)
db.commit()

### LINK_Match 

In [101]:
source_table = "Match"
# drops
sql_drop= f"""DROP TABLE IF EXISTS LINK_{source_table};"""
cursor.execute(sql_drop)

 # creates
sql_create = f"""CREATE TABLE LINK_{source_table}(HK TEXT NOT NULL PRIMARY KEY, HK_away_team TEXT, HK_home_team TEXT, HK_league TEXT, LDTS DATETIME, RS INTEGER);"""
cursor.execute(sql_create)

# inserts
sql_insert = f"""INSERT INTO LINK_{source_table}(HK, HK_away_team, HK_home_team, HK_league, LDTS, RS)
            SELECT 
                season||'_'||away_team_api_id||'_'||home_team_api_id||'_'||league_id||'_'||date||'_'||CAST(1 AS STR),
                away_team_api_id||'_'||CAST(1 AS STR),
                home_team_api_id||'_'||CAST(1 AS STR),
                league_id||'_'||CAST(1 AS STR),
                datetime('now'), 1
            FROM {source_table}"""
cursor.execute(sql_insert)

# converts DF
df = pd.read_sql(f'Select * From LINK_{source_table};', con=db)

# generate hash
df['HK'] = df['HK'].apply(lambda w: data_vault_hash(w))  # data_vault_hash function
df['HK_away_team'] = df['HK_away_team'].apply(lambda w: data_vault_hash(w))
df['HK_home_team'] = df['HK_home_team'].apply(lambda w: data_vault_hash(w))
df['HK_league'] = df['HK_league'].apply(lambda w: data_vault_hash(w))

# load df as a table
df.to_sql(f'LINK_{source_table}', con=db, if_exists='replace',  index=False,
                   dtype={"HK": "TEXT NOT NULL PRIMARY KEY", "HK_away_team":"TEXT", 'HK_home_team':"TEXT", 'HK_league':"TEXT", "LDTS": "DATETIME", "RS": "INTEGER"})

cursor.execute(f"""CREATE TABLE new_LINK_{source_table}(HK TEXT NOT NULL PRIMARY KEY, HK_away_team TEXT, HK_home_team TEXT, HK_league TEXT, LDTS DATETIME, RS INTEGER,
                                 FOREIGN KEY (HK_away_team) REFERENCES HUB_team (away_team_api_id),
                                 FOREIGN KEY (HK_home_team) REFERENCES HUB_team (home_team_api_id),
                                 FOREIGN KEY (HK_league) REFERENCES HUB_league (league_id));""")

cursor.execute(f"INSERT INTO new_LINK_{source_table} SELECT * FROM LINK_{source_table};")

cursor.execute(f"DROP TABLE LINK_{source_table};")
cursor.execute(f"ALTER TABLE new_LINK_{source_table} RENAME TO LINK_{source_table};")

df_LINK = pd.read_sql(f'Select * From LINK_{source_table};', con=db)
df_LINK.head()

,HK,HK_away_team,HK_home_team,HK_league,LDTS,RS
0,975b30a766e01a9cf33f9358aeb1075d,cf0edb2d39ab5700564c89e812f63db1,35864c52d3d3a5cb853f180500483480,ec308451c1d095c528cfa3c009ea7235,2021-02-10 10:42:24,1
1,b2dadc17e61cd5ae4184e9f214db709b,847d83a8aa5e4694d0f20309d18d4410,e29eebd093f578d532f9999538481eb7,ec308451c1d095c528cfa3c009ea7235,2021-02-10 10:42:24,1
2,a3f9b88cb1376d0cb068e04b12fbfb3b,98422aeb52fc204083f867af3a569758,235b5f87cec01bdc8a19786fe03a9751,ec308451c1d095c528cfa3c009ea7235,2021-02-10 10:42:24,1
3,2a2e8db6d8cbf0f39b05f26650827779,60a1ea9ff920223c747f6e4594daceb7,307b3fd8ae2165be5205d787b3ee287d,ec308451c1d095c528cfa3c009ea7235,2021-02-10 10:42:24,1
4,2974d8e1eb83f76c233313d5d288ab7e,7e8159fb3c3c04e179a855f53e629951,ad4667071684915afb120a03b1cafdd1,ec308451c1d095c528cfa3c009ea7235,2021-02-10 10:42:24,1


In [87]:
df_HUB = pd.read_sql(f'Select * From HUB_Player;', con=db)
df_HUB.head()

,HK,BK,LDTS,RS
0,1adb0c2cf78aa85dd55a30506d9695ba,2625,2021-02-10 08:24:16,1
1,04b1883afc7fd50e7ff15a3b0f5a87b8,2752,2021-02-10 08:24:16,1
2,3c1f0cffb54784fffcfad0d9d2819910,2768,2021-02-10 08:24:16,1
3,8bf1b8616904be9aee4364ae18653479,2770,2021-02-10 08:24:16,1
4,5793dcf6b65b060e64fec83607281293,2790,2021-02-10 08:24:16,1


In [89]:
df_player = pd.read_sql(f'Select * From Player;', con=db)
df_player.sort_values(by="player_api_id")

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
8359,8372,2625,"Patryk Rachwal,18",148544,1981-01-27 00:00:00,175.26,154
2636,2639,2752,Diego Mainz,135819,1982-12-29 00:00:00,187.96,174
5312,5319,2768,Jose Dorado,110019,1982-07-10 00:00:00,180.34,154
4335,4339,2770,Ignacio Gonzalez,182861,1982-05-14 00:00:00,180.34,161
305,306,2790,Alberto Rey,110809,1974-02-15 00:00:00,175.26,163
...,...,...,...,...,...,...,...
3978,3982,744907,Greg Taylor,226508,1997-11-05 00:00:00,172.72,137
3484,3488,746419,Francisco Afonso,233930,1997-04-24 00:00:00,175.26,152
3603,3607,748432,Fynn Arkenberg,233969,1996-03-04 00:00:00,187.96,176
8827,8841,750435,Rees Greenwood,225462,1996-01-20 00:00:00,172.72,150


### LINK_Match_Player

In [48]:
pd.set_option("display.max_column", None)

In [229]:
# converts DF
df = pd.read_sql(f'Select * From Match;', con=db)
df.head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,1.73,3.40,5.00,1.75,3.35,4.20,1.85,3.2,3.5,1.80,3.3,3.75,NaN,NaN,NaN,1.70,3.30,4.33,1.90,3.3,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,1.95,3.20,3.60,1.80,3.30,3.95,1.90,3.2,3.5,1.90,3.2,3.50,NaN,NaN,NaN,1.83,3.30,3.60,1.95,3.3,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,2.38,3.30,2.75,2.40,3.30,2.55,2.60,3.1,2.3,2.50,3.2,2.50,NaN,NaN,NaN,2.50,3.25,2.40,2.63,3.3,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,1.44,3.75,7.50,1.40,4.00,6.80,1.40,3.9,6.0,1.44,3.6,6.50,NaN,NaN,NaN,1.44,3.75,6.00,1.44,4.0,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,5.00,3.50,1.65,5.00,3.50,1.60,4.00,3.3,1.7,4.00,3.4,1.72,NaN,NaN,NaN,4.20,3.40,1.70,4.50,3.5,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67


In [230]:
column_list = ['league_id', 'match_api_id', 'season', 'date', 'home_team_api_id', 'away_team_api_id', 'home_player_1', "home_player_2", "home_player_3", "home_player_4",
               "home_player_5", "home_player_6", "home_player_7", "home_player_8",	"home_player_9", "home_player_10", "home_player_11", "away_player_1",
               "away_player_2", "away_player_3", "away_player_4", "away_player_5", "away_player_6", "away_player_7", "away_player_8", "away_player_9",
               "away_player_10", "away_player_11"]

In [231]:
player_list = ['home_player_1', "home_player_2", "home_player_3", "home_player_4",
               "home_player_5", "home_player_6", "home_player_7", "home_player_8",	"home_player_9", "home_player_10", "home_player_11", "away_player_1",
               "away_player_2", "away_player_3", "away_player_4", "away_player_5", "away_player_6", "away_player_7", "away_player_8", "away_player_9",
               "away_player_10", "away_player_11"]

In [232]:
df_link = df.loc[:, column_list]
df_link.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25979 entries, 0 to 25978
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   league_id         25979 non-null  int64  
 1   match_api_id      25979 non-null  int64  
 2   season            25979 non-null  object 
 3   date              25979 non-null  object 
 4   home_team_api_id  25979 non-null  int64  
 5   away_team_api_id  25979 non-null  int64  
 6   home_player_1     24755 non-null  float64
 7   home_player_2     24664 non-null  float64
 8   home_player_3     24698 non-null  float64
 9   home_player_4     24656 non-null  float64
 10  home_player_5     24663 non-null  float64
 11  home_player_6     24654 non-null  float64
 12  home_player_7     24752 non-null  float64
 13  home_player_8     24670 non-null  float64
 14  home_player_9     24706 non-null  float64
 15  home_player_10    24543 non-null  float64
 16  home_player_11    24424 non-null  float6

In [233]:
df_link.head()

,league_id,match_api_id,season,date,home_team_api_id,away_team_api_id,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11
0,1,492473,2008/2009,2008-08-17 00:00:00,9987,9993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,492474,2008/2009,2008-08-16 00:00:00,10000,9994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,492475,2008/2009,2008-08-16 00:00:00,9984,8635,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,492476,2008/2009,2008-08-17 00:00:00,9991,9998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,492477,2008/2009,2008-08-16 00:00:00,7947,9985,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [234]:
df_link[df.home_team_api_id == 9987].head(5)

,league_id,match_api_id,season,date,home_team_api_id,away_team_api_id,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11
0,1,492473,2008/2009,2008-08-17 00:00:00,9987,9993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,1,492583,2008/2009,2008-11-15 00:00:00,9987,9999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,1,492651,2008/2009,2008-11-29 00:00:00,9987,9984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,1,492713,2008/2009,2008-12-13 00:00:00,9987,9986,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93,1,492805,2008/2009,2009-01-24 00:00:00,9987,9998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [235]:
df_melt = df.melt(id_vars=["match_api_id","season","date", "home_team_api_id", "away_team_api_id", "league_id"], value_vars=player_list, value_name="player_api_id", var_name='Player')
df_melt.sort_values(by=["season", "match_api_id"], ignore_index=True, inplace=True)
df_melt.head(5)
# season||'_'||away_team_api_id||'_'||home_team_api_id||'_'||league_id||'_'||date||'_'||CAST(1 AS STR),  --> HK_link_match
# player_api_id||'_'||CAST(1 AS STR)

,match_api_id,season,date,home_team_api_id,away_team_api_id,league_id,Player,player_api_id
0,483129,2008/2009,2008-08-09 00:00:00,8583,9830,4769,home_player_1,40714.0
1,483129,2008/2009,2008-08-09 00:00:00,8583,9830,4769,home_player_2,11323.0
2,483129,2008/2009,2008-08-09 00:00:00,8583,9830,4769,home_player_3,30823.0
3,483129,2008/2009,2008-08-09 00:00:00,8583,9830,4769,home_player_4,26136.0
4,483129,2008/2009,2008-08-09 00:00:00,8583,9830,4769,home_player_5,39996.0


In [236]:
# creates LDTS
now = dt.datetime.now()
load_time = now.strftime("%Y/%m/%d %H:%M:%S")  # LDTS
df_melt["LDTS"] = load_time

In [237]:
# creates RS column
df_melt["RS"] = 1

In [238]:
# creates HK_source_table with hash
df_melt["HK_link_match"] = df_melt.apply(lambda row : str(row["season"])+ '_' + str(row["away_team_api_id"])+'_'+str(row["home_team_api_id"])+'_'+str(row["league_id"])+'_'+str(row["date"])+'_'+str(row["RS"]), axis=1)
df_melt["HK_player"] = df_melt.apply(lambda row: '0'+'_'+str(row["RS"]) if str(row["player_api_id"]) == np.nan else str(row["player_api_id"])+'_'+str(row["RS"]), axis=1)

In [190]:
#df_melt[df_melt.player_api_id == 'NaN']
df_melt[df_melt.player_api_id.isnull()].head(2)

,match_api_id,season,date,home_team_api_id,away_team_api_id,league_id,Player,player_api_id,LDTS,RS,HK_link_match,HK_player
12,483129,2008/2009,2008-08-09 00:00:00,8583,9830,4769,away_player_2,NaN,2021/02/10 12:40:04,1,2008/2009_9830_8583_4769_2008-08-09 00:00:00_1,nan_1
66,483132,2008/2009,2008-08-09 00:00:00,8682,8689,4769,home_player_1,NaN,2021/02/10 12:40:04,1,2008/2009_8689_8682_4769_2008-08-09 00:00:00_1,nan_1


In [239]:
df_melt["HK_player"] = df_melt.apply(lambda row: row["HK_player"].replace(".0", ""), axis=1)

In [240]:
df_melt.head(2)

,match_api_id,season,date,home_team_api_id,away_team_api_id,league_id,Player,player_api_id,LDTS,RS,HK_link_match,HK_player
0,483129,2008/2009,2008-08-09 00:00:00,8583,9830,4769,home_player_1,40714.0,2021/02/10 15:25:32,1,2008/2009_9830_8583_4769_2008-08-09 00:00:00_1,40714_1
1,483129,2008/2009,2008-08-09 00:00:00,8583,9830,4769,home_player_2,11323.0,2021/02/10 15:25:32,1,2008/2009_9830_8583_4769_2008-08-09 00:00:00_1,11323_1


In [241]:
# creates HK column for LINK_match_player
df_melt["HK"] = df_melt.apply(lambda row : row["Player"]+'_'+row["HK_link_match"]+ '_' + row["HK_player"] + '_' + str(row["RS"]), axis=1)

In [242]:
df_melt.head(2)

,match_api_id,season,date,home_team_api_id,away_team_api_id,league_id,Player,player_api_id,LDTS,RS,HK_link_match,HK_player,HK
0,483129,2008/2009,2008-08-09 00:00:00,8583,9830,4769,home_player_1,40714.0,2021/02/10 15:25:32,1,2008/2009_9830_8583_4769_2008-08-09 00:00:00_1,40714_1,home_player_1_2008/2009_9830_8583_4769_2008-08...
1,483129,2008/2009,2008-08-09 00:00:00,8583,9830,4769,home_player_2,11323.0,2021/02/10 15:25:32,1,2008/2009_9830_8583_4769_2008-08-09 00:00:00_1,11323_1,home_player_2_2008/2009_9830_8583_4769_2008-08...


In [243]:
df_melt["HK"] = df_melt.apply(lambda row: data_vault_hash(row["HK"]), axis=1)

In [244]:
df_melt["HK_link_match"] = df_melt.apply(lambda row: data_vault_hash(row["HK_link_match"]), axis=1)

In [245]:
df_melt["HK_player"] = df_melt.apply(lambda row: data_vault_hash(row["HK_player"]), axis=1)

In [246]:
df_melt.head(2)

,match_api_id,season,date,home_team_api_id,away_team_api_id,league_id,Player,player_api_id,LDTS,RS,HK_link_match,HK_player,HK
0,483129,2008/2009,2008-08-09 00:00:00,8583,9830,4769,home_player_1,40714.0,2021/02/10 15:25:32,1,891cbcf3d6d597aa2d46c0ca3b402c7f,6e209fc7e87fcbc1698af5ad333cb0a6,c7ce5488bcd33a0b78ecff3ba51483c8
1,483129,2008/2009,2008-08-09 00:00:00,8583,9830,4769,home_player_2,11323.0,2021/02/10 15:25:32,1,891cbcf3d6d597aa2d46c0ca3b402c7f,1d1daccdb4f9a49528d51a2c184366f6,82df60219c8793f5053ae3cf33a6bd4c


In [247]:
df = pd.read_sql(f'Select * From HUB_player;', con=db)
df[df.BK == 40714].head()

,HK,BK,LDTS,RS
3939,6e209fc7e87fcbc1698af5ad333cb0a6,40714,2021-02-10 08:24:16,1


In [248]:
df_match_player = df_melt[["HK", "HK_link_match", "HK_player", "LDTS", "RS"]]
df_match_player.head(2)

,HK,HK_link_match,HK_player,LDTS,RS
0,c7ce5488bcd33a0b78ecff3ba51483c8,891cbcf3d6d597aa2d46c0ca3b402c7f,6e209fc7e87fcbc1698af5ad333cb0a6,2021/02/10 15:25:32,1
1,82df60219c8793f5053ae3cf33a6bd4c,891cbcf3d6d597aa2d46c0ca3b402c7f,1d1daccdb4f9a49528d51a2c184366f6,2021/02/10 15:25:32,1


In [249]:
df_match_player.shape

(571538, 5)

In [250]:
# load df_match_player as a table
df_match_player.to_sql('LINK_Match_Player', con=db, if_exists='replace',  index=False,
                   dtype={"HK": "TEXT NOT NULL PRIMARY KEY", "HK_link_match":"TEXT", 'HK_player':"TEXT", "LDTS": "DATETIME", "RS": "INTEGER"})

In [251]:
df_LINK_match_player = pd.read_sql(f'Select * From LINK_Match_Player;', con=db)
df_LINK_match_player.head()

,HK,HK_link_match,HK_player,LDTS,RS
0,c7ce5488bcd33a0b78ecff3ba51483c8,891cbcf3d6d597aa2d46c0ca3b402c7f,6e209fc7e87fcbc1698af5ad333cb0a6,2021/02/10 15:25:32,1
1,82df60219c8793f5053ae3cf33a6bd4c,891cbcf3d6d597aa2d46c0ca3b402c7f,1d1daccdb4f9a49528d51a2c184366f6,2021/02/10 15:25:32,1
2,26731bb4667b6cfbd10da1d486dde972,891cbcf3d6d597aa2d46c0ca3b402c7f,be7627ceb7b37620b3135b57bf755ffe,2021/02/10 15:25:32,1
3,237d3ea910b6cecab389ad5b4c5e9dd7,891cbcf3d6d597aa2d46c0ca3b402c7f,0af66d2d966d1801b95ed5ba3fdf493c,2021/02/10 15:25:32,1
4,f27b046d01c0a7839deada4c2b5af39a,891cbcf3d6d597aa2d46c0ca3b402c7f,77e1df8c3b241de0efd70400f400634e,2021/02/10 15:25:32,1


In [252]:
cursor.execute(f"DROP TABLE IF EXISTS new_LINK_Match_Player;")

In [253]:
cursor.execute(f"""CREATE TABLE new_LINK_Match_Player(HK TEXT NOT NULL PRIMARY KEY, HK_link_match TEXT, HK_player TEXT, LDTS DATETIME, RS INTEGER,
                                 FOREIGN KEY (HK_link_match) REFERENCES LINK_Match (HK),
                                 FOREIGN KEY (HK_player) REFERENCES HUB_Player (HK));""")

In [254]:
cursor.execute(f"INSERT INTO new_LINK_Match_Player SELECT * FROM LINK_Match_Player;")

In [255]:
cursor.execute(f"DROP TABLE LINK_Match_Player;")

In [256]:
cursor.execute(f"ALTER TABLE new_LINK_Match_Player RENAME TO LINK_Match_Player;")

In [257]:
df_LINK_match_player = pd.read_sql(f'Select * From LINK_Match_Player;', con=db)
df_LINK_match_player.head()

,HK,HK_link_match,HK_player,LDTS,RS
0,c7ce5488bcd33a0b78ecff3ba51483c8,891cbcf3d6d597aa2d46c0ca3b402c7f,6e209fc7e87fcbc1698af5ad333cb0a6,2021/02/10 15:25:32,1
1,82df60219c8793f5053ae3cf33a6bd4c,891cbcf3d6d597aa2d46c0ca3b402c7f,1d1daccdb4f9a49528d51a2c184366f6,2021/02/10 15:25:32,1
2,26731bb4667b6cfbd10da1d486dde972,891cbcf3d6d597aa2d46c0ca3b402c7f,be7627ceb7b37620b3135b57bf755ffe,2021/02/10 15:25:32,1
3,237d3ea910b6cecab389ad5b4c5e9dd7,891cbcf3d6d597aa2d46c0ca3b402c7f,0af66d2d966d1801b95ed5ba3fdf493c,2021/02/10 15:25:32,1
4,f27b046d01c0a7839deada4c2b5af39a,891cbcf3d6d597aa2d46c0ca3b402c7f,77e1df8c3b241de0efd70400f400634e,2021/02/10 15:25:32,1


In [259]:
get_table_names(db)

['Player_Attributes',
 'Player',
 'Match',
 'League',
 'Country',
 'Team',
 'Team_Attributes',
 'HUB_player',
 'HUB_league',
 'HUB_team',
 'LINK_Match',
 'LINK_Match_Player']

In [260]:
db.commit()

In [261]:
db.close()

### HSAT TABELLEN

Sometimes, we do not know in advance the number of arguments that will be passed into a function. Python allows us to handle this kind of situation through function calls with an arbitrary number of arguments. We have arbitrary numbers of columns (descriptive Attribute) for every single HSAT.

In the function definition, we use an asterisk (*) before the parameter name to denote this kind of argument.

In [562]:
def hsat_table(source_table, db, att_list: list, table_name=None, date=False, unique=False):  # args can be either list of attributes or only attributes.
    """takes source table, definiert a DF, create a new DF and save to SQlite as a HSAT table"""
    if type(source_table) == str:
        # converts DF
        df = pd.read_sql(f'Select * From {source_table};', con=db)
        
        # finds primary key
        primary_key = get_column_names(source_table, db)[1]
        if unique == True:
            primary_key2 = get_primary_key2(source_table, db)
        
        # creates new DF from source df 
        df_2 = df[primary_key].to_frame(name=f"HK_{source_table}")   

        # creates LDTS and EDTS(12 hours later) column
        now = dt.datetime.now()
        load_time = now.strftime("%Y/%m/%d %H:%M:%S")  # LDTS
        df_2["LDTS"] = load_time

        end = now + dt.timedelta(hours=12)
        end_time = end.strftime("%Y/%m/%d %H:%M:%S")  # EDTS
        df_2["EDTS"] = end_time

        # creates RS column
        df_2["RS"] = 1

        # creates HK_source_table column with hash
        df_2[f"HK_{source_table}"] = df_2.apply(lambda row : str(row[f"HK_{source_table}"])+ '_' + str(row["RS"]), axis=1)
        df_2[f"HK_{source_table}"] = df_2.apply(lambda row: data_vault_hash(row[f"HK_{source_table}"]), axis=1)

        # creates HK column for HSAT with hash
        if date==False and unique==False:
            df_2["HK"] = df_2.apply(lambda row : str(row[f"HK_{source_table}"])+ '_' + str(row["RS"])+'_'+str(row["LDTS"]), axis=1)
            df_2["HK"] = df_2.apply(lambda row: data_vault_hash(row["HK"]), axis=1)
        if date==True and unique==False:
            df_2["date"] = df["date"]
            df_2["HK"] = df_2.apply(lambda row : str(row[f"HK_{source_table}"])+ '_' + str(row["RS"])+'_'+str(row["LDTS"]) + '_' + str(row["date"]), axis=1)
            df_2["HK"] = df_2.apply(lambda row: data_vault_hash(row["HK"]), axis=1)
        if date==True and unique==True:
            df_2["date"] = df["date"]
            df_2[primary_key2] = df[primary_key2]
            df_2["HK"] = df_2.apply(lambda row : str(row[f"HK_{source_table}"])+ '_' + str(row["RS"])+'_'+str(row["LDTS"]) + '_' + str(row["date"]) + '_' + str(row[primary_key2]), axis=1)
            df_2["HK"] = df_2.apply(lambda row: data_vault_hash(row["HK"]), axis=1)

        # create attribute columns
        for attribute in att_list:
            df_2[attribute] = df[attribute]

        # create Hash_Diff
        att_num = len(att_list)
        df_2["HD"] = (df_2.iloc[:,-1]).astype(str)

        for i in range(3, att_num+1):
            df_2["HD"] = df_2["HD"] +"_"+ (df_2.iloc[:,-i]).astype(str)
       
        df_2["HD"] = df_2.apply(lambda row: data_vault_hash(row["HD"]), axis=1)

        # column allocation (HK -> 1, HD -> 5)
        first_col = df_2.pop("HK")
        df_2.insert(0, "HK", first_col)
        fifth_col = df_2.pop("HD")
        df_2.insert(5, "HD", fifth_col)
    
    else:  # if source table is a DataFrame 
        
        primary_key = source_table.columns[1]
        if unique == True:
            primary_key2 = source_table.columns[0]
        source_df = source_table.copy()
        source_table = table_name
        
        # creates new DF from source df 
        df_2 = source_df[primary_key].to_frame(name=f"HK_{source_table}")
        
        # creates LDTS and EDTS(12 hours later) column
        now = dt.datetime.now()
        load_time = now.strftime("%Y/%m/%d %H:%M:%S")  # LDTS
        df_2["LDTS"] = load_time

        end = now + dt.timedelta(hours=12)
        end_time = end.strftime("%Y/%m/%d %H:%M:%S")  # EDTS
        df_2["EDTS"] = end_time

        # creates RS column
        df_2["RS"] = 1
        
        # creates HK_source_table column with hash
        df_2[f"HK_{source_table}"] = df_2.apply(lambda row : str(row[f"HK_{source_table}"])+ '_' + str(row["RS"]), axis=1)
        df_2[f"HK_{source_table}"] = df_2.apply(lambda row: data_vault_hash(row[f"HK_{source_table}"]), axis=1)

        # creates HK column for HSAT with hash
        if date==False and unique==False:
            df_2["HK"] = df_2.apply(lambda row : str(row[f"HK_{source_table}"])+ '_' + str(row["RS"])+'_'+str(row["LDTS"]), axis=1)
            df_2["HK"] = df_2.apply(lambda row: data_vault_hash(row["HK"]), axis=1)
        if date==True and unique==False:
            df_2["date"] = source_df["date"]
            df_2["HK"] = df_2.apply(lambda row : str(row[f"HK_{source_table}"])+ '_' + str(row["RS"])+'_'+str(row["LDTS"]) + '_' + str(row["date"]), axis=1)
            df_2["HK"] = df_2.apply(lambda row: data_vault_hash(row["HK"]), axis=1)
        if date==True and unique==True:
            df_2["date"] = source_df["date"]
            df_2[primary_key2] = source_df[primary_key2]
            df_2["HK"] = df_2.apply(lambda row : str(row[f"HK_{source_table}"])+ '_' + str(row["RS"])+'_'+str(row["LDTS"]) + '_' + str(row["date"]) + '_' + str(row[primary_key2]), axis=1)
            df_2["HK"] = df_2.apply(lambda row: data_vault_hash(row["HK"]), axis=1)

        # create attribute columns
        for attribute in att_list:
            df_2[attribute] = source_df[attribute]
                
        # create Hash_Diff
        att_num = len(att_list)
        df_2["HD"] = (df_2.iloc[:,-1]).astype(str)

        for i in range(3, att_num+2):
            df_2["HD"] = df_2["HD"] +"_"+ (df_2.iloc[:,-i]).astype(str)         
    
        df_2["HD"] = df_2.apply(lambda row: data_vault_hash(row["HD"]), axis=1)

        # column allocation (HK -> 1, HD -> 5)
        first_col = df_2.pop("HK")
        df_2.insert(0, "HK", first_col)
        fifth_col = df_2.pop("HD")
        df_2.insert(5, "HD", fifth_col)
    
    #load df as a table to SQL
    df_2.to_sql(f'HSAT_{source_table}', con=db, if_exists='replace',  index=False,
                dtype={"HK": "TEXT NOT NULL PRIMARY KEY", f"HK_{source_table}":"TEXT", "LDTS": "DATETIME", "EDTS": "DATETIME",
                        "RS": "INTEGER", "HD": "TEXT"})
    
    df_HSAT = pd.read_sql(f'Select * From HSAT_{source_table};', con=db)
    return df_HSAT.head(7)

### HSAT_league

In [484]:
# read source table
df_league = pd.read_sql(f'Select * From league;', con=db)
df_league.head(2)

,id,country_id,name
0,1,1,Belgium Jupiler League
1,1729,1729,England Premier League


In [486]:
# generates HSAT_league table
hsat_table("league", db, ['name'])

,HK,HK_league,LDTS,EDTS,RS,HD,name
0,735687ab3ff241def7151c965937aac4,ec308451c1d095c528cfa3c009ea7235,2021/02/10 22:02:55,2021/02/11 10:02:55,1,fbf25806aa504e04f7e71972a0866213,Belgium Jupiler League
1,ac64f117f5d4da15646b75997400ddba,0bdc65e4364db54d79b628670f347490,2021/02/10 22:02:55,2021/02/11 10:02:55,1,eb389b2404d2db2fbd886c352e0bff45,England Premier League
2,81122954308daccf1a6eb09aa4e77152,248eeffc41601122c9cbcb4ff65441a6,2021/02/10 22:02:55,2021/02/11 10:02:55,1,84c549e9b050a56ede5c8eab7841be83,France Ligue 1
3,f8d3b2d0a2d308e0c9c634fc3e9bc5f4,e0bd49f6b7486603d558fc3a81018a9c,2021/02/10 22:02:55,2021/02/11 10:02:55,1,2fe34a8347cdad3f7de05bb205bee634,Germany 1. Bundesliga
4,fa96be84581b8f014a9e2c29b55cbf21,2963f320cccf82d389a3b13ac26373a7,2021/02/10 22:02:55,2021/02/11 10:02:55,1,97d219b01e9584d3b244108a697c61cf,Italy Serie A


In [487]:
# read HUB table
df_HUB_league = pd.read_sql(f'Select * From HUB_league;', con=db)
df_HUB_league.head(2)

,HK,BK,LDTS,RS
0,ec308451c1d095c528cfa3c009ea7235,1,2021-02-10 08:24:16,1
1,0bdc65e4364db54d79b628670f347490,1729,2021-02-10 08:24:16,1


##### REFERENCE TO HUB_League 

In [616]:
source_table = "league"

In [617]:
get_column_names(f"HSAT_{source_table}", db) # [5:]

['HK', 'HK_league', 'LDTS', 'EDTS', 'RS', 'HD', 'name']

In [618]:
cursor.execute(f"DROP TABLE IF EXISTS new_HSAT_{source_table};")
cursor.execute(f"""CREATE TABLE new_HSAT_{source_table}(HK TEXT NOT NULL PRIMARY KEY, HK_{source_table} TEXT, LDTS DATETIME, EDTS DATETIME,RS INTEGER, 'HD' TEXT,
                'name' TEXT, FOREIGN KEY (HK_{source_table}) REFERENCES HUB_{source_table} ("HK"));""")

cursor.execute(f"INSERT INTO new_HSAT_{source_table} SELECT * FROM HSAT_{source_table};")

cursor.execute(f"DROP TABLE HSAT_{source_table};")
cursor.execute(f"ALTER TABLE new_HSAT_{source_table} RENAME TO HSAT_{source_table};")

df_HSAT = pd.read_sql(f'Select * From HSAT_{source_table};', con=db)
df_HSAT.head()

,HK,HK_league,LDTS,EDTS,RS,HD,name
0,735687ab3ff241def7151c965937aac4,ec308451c1d095c528cfa3c009ea7235,2021/02/10 22:02:55,2021/02/11 10:02:55,1,fbf25806aa504e04f7e71972a0866213,Belgium Jupiler League
1,ac64f117f5d4da15646b75997400ddba,0bdc65e4364db54d79b628670f347490,2021/02/10 22:02:55,2021/02/11 10:02:55,1,eb389b2404d2db2fbd886c352e0bff45,England Premier League
2,81122954308daccf1a6eb09aa4e77152,248eeffc41601122c9cbcb4ff65441a6,2021/02/10 22:02:55,2021/02/11 10:02:55,1,84c549e9b050a56ede5c8eab7841be83,France Ligue 1
3,f8d3b2d0a2d308e0c9c634fc3e9bc5f4,e0bd49f6b7486603d558fc3a81018a9c,2021/02/10 22:02:55,2021/02/11 10:02:55,1,2fe34a8347cdad3f7de05bb205bee634,Germany 1. Bundesliga
4,fa96be84581b8f014a9e2c29b55cbf21,2963f320cccf82d389a3b13ac26373a7,2021/02/10 22:02:55,2021/02/11 10:02:55,1,97d219b01e9584d3b244108a697c61cf,Italy Serie A


### HSAT_team

In [434]:
# read source table
df_team = pd.read_sql(f'Select * From Team;', con=db)
df_team.head(2)

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673.0,KRC Genk,GEN
1,2,9993,675.0,Beerschot AC,BAC


In [435]:
# read source table
df_team_att = pd.read_sql(f'Select * From Team_Attributes;', con=db)
df_team_att.head(2)

,id,team_fifa_api_id,team_api_id,date,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribbling,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,buildUpPlayPositioningClass,chanceCreationPassing,chanceCreationPassingClass,chanceCreationCrossing,chanceCreationCrossingClass,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass
0,1,434,9930,2010-02-22 00:00:00,60,Balanced,NaN,Little,50,Mixed,Organised,60,Normal,65,Normal,55,Normal,Organised,50,Medium,55,Press,45,Normal,Cover
1,2,434,9930,2014-09-19 00:00:00,52,Balanced,48.0,Normal,56,Mixed,Organised,54,Normal,63,Normal,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover


In [436]:
df_team_att = df_team_att.merge(df_team, how="left", on="team_api_id")
df_team_att.head(2)

,id_x,team_fifa_api_id_x,team_api_id,date,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribbling,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,buildUpPlayPositioningClass,chanceCreationPassing,chanceCreationPassingClass,chanceCreationCrossing,chanceCreationCrossingClass,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass,id_y,team_fifa_api_id_y,team_long_name,team_short_name
0,1,434,9930,2010-02-22 00:00:00,60,Balanced,NaN,Little,50,Mixed,Organised,60,Normal,65,Normal,55,Normal,Organised,50,Medium,55,Press,45,Normal,Cover,49119,434.0,FC Aarau,AAR
1,2,434,9930,2014-09-19 00:00:00,52,Balanced,48.0,Normal,56,Mixed,Organised,54,Normal,63,Normal,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover,49119,434.0,FC Aarau,AAR


In [437]:
# column allocation (team_api_id -> 0)
first_col = df_team_att.pop("team_api_id")
df_team_att.insert(1, "team_api_id", first_col)
team = df_team_att.copy()
team.head()

,id_x,team_api_id,team_fifa_api_id_x,date,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribbling,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,buildUpPlayPositioningClass,chanceCreationPassing,chanceCreationPassingClass,chanceCreationCrossing,chanceCreationCrossingClass,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass,id_y,team_fifa_api_id_y,team_long_name,team_short_name
0,1,9930,434,2010-02-22 00:00:00,60,Balanced,NaN,Little,50,Mixed,Organised,60,Normal,65,Normal,55,Normal,Organised,50,Medium,55,Press,45,Normal,Cover,49119,434.0,FC Aarau,AAR
1,2,9930,434,2014-09-19 00:00:00,52,Balanced,48.0,Normal,56,Mixed,Organised,54,Normal,63,Normal,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover,49119,434.0,FC Aarau,AAR
2,3,9930,434,2015-09-10 00:00:00,47,Balanced,41.0,Normal,54,Mixed,Organised,54,Normal,63,Normal,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover,49119,434.0,FC Aarau,AAR
3,4,8485,77,2010-02-22 00:00:00,70,Fast,NaN,Little,70,Long,Organised,70,Risky,70,Lots,70,Lots,Organised,60,Medium,70,Double,70,Wide,Cover,39393,77.0,Aberdeen,ABE
4,5,8485,77,2011-02-22 00:00:00,47,Balanced,NaN,Little,52,Mixed,Organised,53,Normal,48,Normal,52,Normal,Organised,47,Medium,47,Press,52,Normal,Cover,39393,77.0,Aberdeen,ABE


In [490]:
team.columns

Index(['id_x', 'team_api_id', 'team_fifa_api_id_x', 'date', 'buildUpPlaySpeed',
       'buildUpPlaySpeedClass', 'buildUpPlayDribbling',
       'buildUpPlayDribblingClass', 'buildUpPlayPassing',
       'buildUpPlayPassingClass', 'buildUpPlayPositioningClass',
       'chanceCreationPassing', 'chanceCreationPassingClass',
       'chanceCreationCrossing', 'chanceCreationCrossingClass',
       'chanceCreationShooting', 'chanceCreationShootingClass',
       'chanceCreationPositioningClass', 'defencePressure',
       'defencePressureClass', 'defenceAggression', 'defenceAggressionClass',
       'defenceTeamWidth', 'defenceTeamWidthClass', 'defenceDefenderLineClass',
       'id_y', 'team_fifa_api_id_y', 'team_long_name', 'team_short_name'],
      dtype='object')

In [491]:
team.columns[0]

'id_x'

In [493]:
arg_list_team = list(team.columns[3:-4])
arg_list_team.append("team_long_name")
arg_list_team

['date',
 'buildUpPlaySpeed',
 'buildUpPlaySpeedClass',
 'buildUpPlayDribbling',
 'buildUpPlayDribblingClass',
 'buildUpPlayPassing',
 'buildUpPlayPassingClass',
 'buildUpPlayPositioningClass',
 'chanceCreationPassing',
 'chanceCreationPassingClass',
 'chanceCreationCrossing',
 'chanceCreationCrossingClass',
 'chanceCreationShooting',
 'chanceCreationShootingClass',
 'chanceCreationPositioningClass',
 'defencePressure',
 'defencePressureClass',
 'defenceAggression',
 'defenceAggressionClass',
 'defenceTeamWidth',
 'defenceTeamWidthClass',
 'defenceDefenderLineClass',
 'team_long_name']

In [518]:
# generates HSAT_team table
hsat_table(team, db, arg_list_team, table_name="team", date=True, unique=True)  # we need date and unique arguments for Unique HashKey 

,HK,HK_team,LDTS,EDTS,RS,HD,date,id_x,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribbling,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,buildUpPlayPositioningClass,chanceCreationPassing,chanceCreationPassingClass,chanceCreationCrossing,chanceCreationCrossingClass,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass,team_long_name
0,12d44c33c2e3e88288ecc9c32cf7c14e,df5cc46bd93e9acb5b57135328bd9643,2021/02/10 22:40:15,2021/02/11 10:40:15,1,5d745e23d726d2f4caece54e1622fd7e,2010-02-22 00:00:00,1,60,Balanced,NaN,Little,50,Mixed,Organised,60,Normal,65,Normal,55,Normal,Organised,50,Medium,55,Press,45,Normal,Cover,FC Aarau
1,fc82944b9248da0455102dc4461fddbb,df5cc46bd93e9acb5b57135328bd9643,2021/02/10 22:40:15,2021/02/11 10:40:15,1,7f0ecfd0109ec878fd76c436f36a4e69,2014-09-19 00:00:00,2,52,Balanced,48.0,Normal,56,Mixed,Organised,54,Normal,63,Normal,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover,FC Aarau
2,2847c46a57a046c8379c91becc15ad99,df5cc46bd93e9acb5b57135328bd9643,2021/02/10 22:40:15,2021/02/11 10:40:15,1,4e9010cd958a6ed1a0739b651bc48ca9,2015-09-10 00:00:00,3,47,Balanced,41.0,Normal,54,Mixed,Organised,54,Normal,63,Normal,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover,FC Aarau
3,fdab5cdba7c346ae4e4c4557aa37c193,e54f961aa315c784b1b2db5d4d417707,2021/02/10 22:40:15,2021/02/11 10:40:15,1,5b8cdf73dd4ec0b5021f8e746b84d15d,2010-02-22 00:00:00,4,70,Fast,NaN,Little,70,Long,Organised,70,Risky,70,Lots,70,Lots,Organised,60,Medium,70,Double,70,Wide,Cover,Aberdeen
4,17e6beb331add0595045515ec235bbe6,e54f961aa315c784b1b2db5d4d417707,2021/02/10 22:40:15,2021/02/11 10:40:15,1,58679a19c05c60edb7794b71b2d75ad5,2011-02-22 00:00:00,5,47,Balanced,NaN,Little,52,Mixed,Organised,53,Normal,48,Normal,52,Normal,Organised,47,Medium,47,Press,52,Normal,Cover,Aberdeen


In [498]:
# read team table
df_team = pd.read_sql(f'Select * From team;', con=db)
df_team[df_HUB_team.team_long_name == 'FC Aarau']

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
288,49119,9930,434.0,FC Aarau,AAR


In [500]:
# read HUB_team table
df_HUB_team = pd.read_sql(f'Select * From HUB_team;', con=db)
df_HUB_team[df_HUB_team.BK == 9930]

,HK,BK,LDTS,RS
235,df5cc46bd93e9acb5b57135328bd9643,9930,2021-02-10 08:24:16,1


##### REFERENCE TO HUB_TEAMS 

In [577]:
source_table = "team"

In [584]:
get_column_names(f"HSAT_{source_table}", db) # [5:]

['HK',
 'HK_team',
 'LDTS',
 'EDTS',
 'RS',
 'HD',
 'date',
 'id_x',
 'buildUpPlaySpeed',
 'buildUpPlaySpeedClass',
 'buildUpPlayDribbling',
 'buildUpPlayDribblingClass',
 'buildUpPlayPassing',
 'buildUpPlayPassingClass',
 'buildUpPlayPositioningClass',
 'chanceCreationPassing',
 'chanceCreationPassingClass',
 'chanceCreationCrossing',
 'chanceCreationCrossingClass',
 'chanceCreationShooting',
 'chanceCreationShootingClass',
 'chanceCreationPositioningClass',
 'defencePressure',
 'defencePressureClass',
 'defenceAggression',
 'defenceAggressionClass',
 'defenceTeamWidth',
 'defenceTeamWidthClass',
 'defenceDefenderLineClass',
 'team_long_name']

In [585]:
cursor.execute(f"DROP TABLE IF EXISTS new_HSAT_{source_table};")
cursor.execute(f"""CREATE TABLE new_HSAT_{source_table}(HK TEXT NOT NULL PRIMARY KEY, HK_{source_table} TEXT, LDTS DATETIME, EDTS DATETIME,RS INTEGER, 'HD' TEXT,
 'date' TEXT,
 'id_x' TEXT,
 'buildUpPlaySpeed' TEXT,
 'buildUpPlaySpeedClass' TEXT,
 'buildUpPlayDribbling' TEXT,
 'buildUpPlayDribblingClass' TEXT,
 'buildUpPlayPassing' TEXT,
 'buildUpPlayPassingClass' TEXT,
 'buildUpPlayPositioningClass' TEXT,
 'chanceCreationPassing' TEXT,
 'chanceCreationPassingClass' TEXT,
 'chanceCreationCrossing' TEXT,
 'chanceCreationCrossingClass' TEXT,
 'chanceCreationShooting' TEXT,
 'chanceCreationShootingClass' TEXT,
 'chanceCreationPositioningClass' TEXT,
 'defencePressure' TEXT,
 'defencePressureClass' TEXT,
 'defenceAggression' TEXT,
 'defenceAggressionClass' TEXT,
 'defenceTeamWidth' TEXT,
 'defenceTeamWidthClass' TEXT,
 'defenceDefenderLineClass' TEXT,
 'team_long_name' TEXT,
  FOREIGN KEY (HK_{source_table}) REFERENCES HUB_{source_table} ("HK"));""")

cursor.execute(f"INSERT INTO new_HSAT_{source_table} SELECT * FROM HSAT_{source_table};")

cursor.execute(f"DROP TABLE HSAT_{source_table};")
cursor.execute(f"ALTER TABLE new_HSAT_{source_table} RENAME TO HSAT_{source_table};")

df_HSAT = pd.read_sql(f'Select * From HSAT_{source_table};', con=db)
df_HSAT.head()

,HK,HK_team,LDTS,EDTS,RS,HD,date,id_x,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribbling,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,buildUpPlayPositioningClass,chanceCreationPassing,chanceCreationPassingClass,chanceCreationCrossing,chanceCreationCrossingClass,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass,team_long_name
0,12d44c33c2e3e88288ecc9c32cf7c14e,df5cc46bd93e9acb5b57135328bd9643,2021/02/10 22:40:15,2021/02/11 10:40:15,1,5d745e23d726d2f4caece54e1622fd7e,2010-02-22 00:00:00,1,60,Balanced,None,Little,50,Mixed,Organised,60,Normal,65,Normal,55,Normal,Organised,50,Medium,55,Press,45,Normal,Cover,FC Aarau
1,fc82944b9248da0455102dc4461fddbb,df5cc46bd93e9acb5b57135328bd9643,2021/02/10 22:40:15,2021/02/11 10:40:15,1,7f0ecfd0109ec878fd76c436f36a4e69,2014-09-19 00:00:00,2,52,Balanced,48.0,Normal,56,Mixed,Organised,54,Normal,63,Normal,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover,FC Aarau
2,2847c46a57a046c8379c91becc15ad99,df5cc46bd93e9acb5b57135328bd9643,2021/02/10 22:40:15,2021/02/11 10:40:15,1,4e9010cd958a6ed1a0739b651bc48ca9,2015-09-10 00:00:00,3,47,Balanced,41.0,Normal,54,Mixed,Organised,54,Normal,63,Normal,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover,FC Aarau
3,fdab5cdba7c346ae4e4c4557aa37c193,e54f961aa315c784b1b2db5d4d417707,2021/02/10 22:40:15,2021/02/11 10:40:15,1,5b8cdf73dd4ec0b5021f8e746b84d15d,2010-02-22 00:00:00,4,70,Fast,None,Little,70,Long,Organised,70,Risky,70,Lots,70,Lots,Organised,60,Medium,70,Double,70,Wide,Cover,Aberdeen
4,17e6beb331add0595045515ec235bbe6,e54f961aa315c784b1b2db5d4d417707,2021/02/10 22:40:15,2021/02/11 10:40:15,1,58679a19c05c60edb7794b71b2d75ad5,2011-02-22 00:00:00,5,47,Balanced,None,Little,52,Mixed,Organised,53,Normal,48,Normal,52,Normal,Organised,47,Medium,47,Press,52,Normal,Cover,Aberdeen


### HSAT_player

In [595]:
# read source table
df_player = pd.read_sql(f'Select * From Player;', con=db)
df_player.head(2)

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146


In [596]:
arg_list = df_player.columns[1:]
arg_list

Index(['player_api_id', 'player_name', 'player_fifa_api_id', 'birthday',
       'height', 'weight'],
      dtype='object')

In [597]:
# generates HSAT_player table
hsat_table("player", db, arg_list)

,HK,HK_player,LDTS,EDTS,RS,HD,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,0ebf2c619865fc826ebf2b0c7af32953,36dd02dfaf93f5c8d874cae6cda1e856,2021/02/11 00:12:49,2021/02/11 12:12:49,1,45ce431401f1cbecfd2465a7f80e7cfc,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,616ed0f923c436356765e59c0643ef46,2728a6a677c6c0b01f251f55d7bb5d8e,2021/02/11 00:12:49,2021/02/11 12:12:49,1,76314f2a622cca1517694a29203d7792,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,1bac2ef16401d1805ac8da6c7b6d1e93,c05d390a2d4fcacf4fc462a6844a90b5,2021/02/11 00:12:49,2021/02/11 12:12:49,1,31ef64cdd36d2f69e82aa7b40b9f28d6,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,bf32d306cf501f440280323172da754a,37f11b8ca97c154ef44525fb64a7dce1,2021/02/11 00:12:49,2021/02/11 12:12:49,1,75a44141ea894cbd6fcebcf0e55fb04a,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,8b2247225b6a2c1d995875736e18e07e,b7c33ba6e09622e095fe1bccf61ca21c,2021/02/11 00:12:49,2021/02/11 12:12:49,1,feaa65d87de5ae66f0dcde00e5bc15f2,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154
5,e9a7ad0826295b637513c2d96ca19713,5d6ce1d06b05cb16de6dabb3465f5434,2021/02/11 00:12:49,2021/02/11 12:12:49,1,d94d7e7ab30705cfb42038527b68ff6a,27316,Aaron Hunt,158138,1986-09-04 00:00:00,182.88,161
6,5d0afdfdf4843cc41da7f6ae715df31a,1e8a6755fbcafed796d9c0411868be6b,2021/02/11 00:12:49,2021/02/11 12:12:49,1,6137a6ef21c60ecd22f4a1665901e347,564793,Aaron Kuhl,221280,1996-01-30 00:00:00,172.72,146


In [598]:
# read HUB table
df_HUB_player = pd.read_sql(f'Select * From HUB_player;', con=db)
df_HUB_player[df_HUB_player.BK == 505942]

,HK,BK,LDTS,RS
10452,36dd02dfaf93f5c8d874cae6cda1e856,505942,2021-02-10 08:24:16,1


##### REFERENCE TO HUB_player 

In [619]:
source_table = "player"

In [620]:
get_column_names(f"HSAT_{source_table}", db) # [5:]

['HK',
 'HK_player',
 'LDTS',
 'EDTS',
 'RS',
 'HD',
 'player_api_id',
 'player_name',
 'player_fifa_api_id',
 'birthday',
 'height',
 'weight']

In [622]:
cursor.execute(f"DROP TABLE IF EXISTS new_HSAT_{source_table};")
cursor.execute(f"""CREATE TABLE new_HSAT_{source_table}(HK TEXT NOT NULL PRIMARY KEY, HK_{source_table} TEXT, LDTS DATETIME, EDTS DATETIME, RS INTEGER, 'HD' TEXT,
                'player_api_id' TEXT, 'player_name' TEXT, 'player_fifa_api_id' TEXT, 'birthday' TEXT, 'height' TEXT, 'weight' TEXT, 
                FOREIGN KEY (HK_{source_table}) REFERENCES HUB_{source_table} ("HK"));""")

cursor.execute(f"INSERT INTO new_HSAT_{source_table} SELECT * FROM HSAT_{source_table};")

cursor.execute(f"DROP TABLE HSAT_{source_table};")
cursor.execute(f"ALTER TABLE new_HSAT_{source_table} RENAME TO HSAT_{source_table};")

df_HSAT = pd.read_sql(f'Select * From HSAT_{source_table};', con=db)
df_HSAT.head()

,HK,HK_player,LDTS,EDTS,RS,HD,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,0ebf2c619865fc826ebf2b0c7af32953,36dd02dfaf93f5c8d874cae6cda1e856,2021/02/11 00:12:49,2021/02/11 12:12:49,1,45ce431401f1cbecfd2465a7f80e7cfc,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,616ed0f923c436356765e59c0643ef46,2728a6a677c6c0b01f251f55d7bb5d8e,2021/02/11 00:12:49,2021/02/11 12:12:49,1,76314f2a622cca1517694a29203d7792,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,1bac2ef16401d1805ac8da6c7b6d1e93,c05d390a2d4fcacf4fc462a6844a90b5,2021/02/11 00:12:49,2021/02/11 12:12:49,1,31ef64cdd36d2f69e82aa7b40b9f28d6,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,bf32d306cf501f440280323172da754a,37f11b8ca97c154ef44525fb64a7dce1,2021/02/11 00:12:49,2021/02/11 12:12:49,1,75a44141ea894cbd6fcebcf0e55fb04a,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,8b2247225b6a2c1d995875736e18e07e,b7c33ba6e09622e095fe1bccf61ca21c,2021/02/11 00:12:49,2021/02/11 12:12:49,1,feaa65d87de5ae66f0dcde00e5bc15f2,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154


### HSAT_player_attribute

In [599]:
# read source table
df_player_att = pd.read_sql(f'Select * From Player_Attributes;', con=db)
df_player_att.head(7)

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,71.0,70.0,45.0,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,71.0,70.0,45.0,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,63.0,41.0,45.0,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,43.0,70.0,60.0,43.0,50.0,44.0,38.0,63.0,48.0,60.0,64.0,59.0,46.0,65.0,54.0,58.0,54.0,76.0,34.0,62.0,40.0,44.0,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,43.0,70.0,60.0,43.0,50.0,44.0,38.0,63.0,48.0,60.0,64.0,59.0,46.0,65.0,54.0,58.0,54.0,76.0,34.0,62.0,40.0,44.0,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
5,6,189615,155782,2016-04-21 00:00:00,74.0,76.0,left,high,medium,80.0,53.0,58.0,71.0,40.0,73.0,70.0,69.0,68.0,71.0,79.0,78.0,78.0,67.0,90.0,71.0,85.0,79.0,56.0,62.0,68.0,67.0,60.0,66.0,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0
6,7,189615,155782,2016-04-07 00:00:00,74.0,76.0,left,high,medium,80.0,53.0,58.0,71.0,32.0,73.0,70.0,69.0,68.0,71.0,79.0,78.0,78.0,67.0,90.0,71.0,85.0,79.0,56.0,60.0,68.0,67.0,60.0,66.0,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0


In [600]:
# column allocation (player_api_id -> 1)
first_col = df_player_att.pop("player_api_id")
df_player_att.insert(1, "player_api_id", first_col)
df_player_att.head(2)

,id,player_api_id,player_fifa_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,505942,218353,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,71.0,70.0,45.0,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,505942,218353,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,71.0,70.0,45.0,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0


In [601]:
df_player_att.shape

(183978, 42)

In [602]:
arg_list_att = df_player_att.columns[3:]
arg_list_att

Index(['date', 'overall_rating', 'potential', 'preferred_foot',
       'attacking_work_rate', 'defensive_work_rate', 'crossing', 'finishing',
       'heading_accuracy', 'short_passing', 'volleys', 'dribbling', 'curve',
       'free_kick_accuracy', 'long_passing', 'ball_control', 'acceleration',
       'sprint_speed', 'agility', 'reactions', 'balance', 'shot_power',
       'jumping', 'stamina', 'strength', 'long_shots', 'aggression',
       'interceptions', 'positioning', 'vision', 'penalties', 'marking',
       'standing_tackle', 'sliding_tackle', 'gk_diving', 'gk_handling',
       'gk_kicking', 'gk_positioning', 'gk_reflexes'],
      dtype='object')

In [603]:
pd.set_option("display.max_column", None)

In [604]:
# generates HSAT_player_att table
hsat_table(df_player_att, db, arg_list_att, table_name="player_att", date=True, unique=True)

,HK,HK_player_att,LDTS,EDTS,RS,HD,date,id,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,d97bfcfb02ac4c30963791aae73354e2,36dd02dfaf93f5c8d874cae6cda1e856,2021/02/11 00:15:29,2021/02/11 12:15:29,1,dbf4d28bb8a765a941093977e21f6363,2016-02-18 00:00:00,1,67.0,71.0,right,medium,medium,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,71.0,70.0,45.0,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,525708d4b725b825ae4e56ec2f7471a8,36dd02dfaf93f5c8d874cae6cda1e856,2021/02/11 00:15:29,2021/02/11 12:15:29,1,70f2ce10d8f3e53b2900160809ce001c,2015-11-19 00:00:00,2,67.0,71.0,right,medium,medium,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,71.0,70.0,45.0,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,a17dbb36d53cd7e6ac0ffd474218b1ac,36dd02dfaf93f5c8d874cae6cda1e856,2021/02/11 00:15:29,2021/02/11 12:15:29,1,dcc6fde568a5f2d3dc14d22e7d51bedb,2015-09-21 00:00:00,3,62.0,66.0,right,medium,medium,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,63.0,41.0,45.0,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,111679c8f2baa87961da501441b81656,36dd02dfaf93f5c8d874cae6cda1e856,2021/02/11 00:15:29,2021/02/11 12:15:29,1,b00908236e2703e010df6b0404b2837b,2015-03-20 00:00:00,4,61.0,65.0,right,medium,medium,48.0,43.0,70.0,60.0,43.0,50.0,44.0,38.0,63.0,48.0,60.0,64.0,59.0,46.0,65.0,54.0,58.0,54.0,76.0,34.0,62.0,40.0,44.0,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,a6f5ef166f23e2e434296c609eaf1b81,36dd02dfaf93f5c8d874cae6cda1e856,2021/02/11 00:15:29,2021/02/11 12:15:29,1,e7a1f9b111502f942d3784ddd5725bd8,2007-02-22 00:00:00,5,61.0,65.0,right,medium,medium,48.0,43.0,70.0,60.0,43.0,50.0,44.0,38.0,63.0,48.0,60.0,64.0,59.0,46.0,65.0,54.0,58.0,54.0,76.0,34.0,62.0,40.0,44.0,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
5,0a0c444e47fcd955345cff01d2b32e7e,2728a6a677c6c0b01f251f55d7bb5d8e,2021/02/11 00:15:29,2021/02/11 12:15:29,1,50e7e9340fba85ab418bfb4a84d400c8,2016-04-21 00:00:00,6,74.0,76.0,left,high,medium,80.0,53.0,58.0,71.0,40.0,73.0,70.0,69.0,68.0,71.0,79.0,78.0,78.0,67.0,90.0,71.0,85.0,79.0,56.0,62.0,68.0,67.0,60.0,66.0,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0
6,0218c0c4ee44ddd11f9e8620fdddca28,2728a6a677c6c0b01f251f55d7bb5d8e,2021/02/11 00:15:29,2021/02/11 12:15:29,1,c0057b369dc9bacfb4c313849be3f6e9,2016-04-07 00:00:00,7,74.0,76.0,left,high,medium,80.0,53.0,58.0,71.0,32.0,73.0,70.0,69.0,68.0,71.0,79.0,78.0,78.0,67.0,90.0,71.0,85.0,79.0,56.0,60.0,68.0,67.0,60.0,66.0,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0


In [605]:
# read HUB_player table
df_HUB_player = pd.read_sql(f'Select * From HUB_player;', con=db)
df_HUB_player[df_HUB_player.BK == 505942]

,HK,BK,LDTS,RS
10452,36dd02dfaf93f5c8d874cae6cda1e856,505942,2021-02-10 08:24:16,1


In [606]:
# read HUB_player table
df_HUB_player = pd.read_sql(f'Select * From HUB_player;', con=db)
df_HUB_player[df_HUB_player.BK == 155782]

,HK,BK,LDTS,RS
6714,2728a6a677c6c0b01f251f55d7bb5d8e,155782,2021-02-10 08:24:16,1


##### REFERENCE TO HUB_player 

In [623]:
source_table = "player_att"

In [624]:
get_column_names(f"HSAT_{source_table}", db) # [5:]

['HK',
 'HK_player_att',
 'LDTS',
 'EDTS',
 'RS',
 'HD',
 'date',
 'id',
 'overall_rating',
 'potential',
 'preferred_foot',
 'attacking_work_rate',
 'defensive_work_rate',
 'crossing',
 'finishing',
 'heading_accuracy',
 'short_passing',
 'volleys',
 'dribbling',
 'curve',
 'free_kick_accuracy',
 'long_passing',
 'ball_control',
 'acceleration',
 'sprint_speed',
 'agility',
 'reactions',
 'balance',
 'shot_power',
 'jumping',
 'stamina',
 'strength',
 'long_shots',
 'aggression',
 'interceptions',
 'positioning',
 'vision',
 'penalties',
 'marking',
 'standing_tackle',
 'sliding_tackle',
 'gk_diving',
 'gk_handling',
 'gk_kicking',
 'gk_positioning',
 'gk_reflexes']

In [626]:
cursor.execute(f"DROP TABLE IF EXISTS new_HSAT_{source_table};")
cursor.execute(f"""CREATE TABLE new_HSAT_{source_table}(HK TEXT NOT NULL PRIMARY KEY, HK_player TEXT, LDTS DATETIME, EDTS DATETIME, RS INTEGER, 'HD' TEXT,
                'date' TEXT, 'id' TEXT, 'overall_rating' TEXT, 'potential' TEXT, 'preferred_foot' TEXT, 'attacking_work_rate' TEXT, 'defensive_work_rate' TEXT, 'crossing' TEXT, 'finishing' TEXT, 'heading_accuracy',
                 'short_passing' TEXT, 'volleys' TEXT, 'dribbling' TEXT, 'curve' TEXT, 'free_kick_accuracy' TEXT, 'long_passing' TEXT, 'ball_control' TEXT, 'acceleration' TEXT, 'sprint_speed' TEXT, 'agility' TEXT,
                  'reactions' TEXT, 'balance' TEXT, 'shot_power' TEXT, 'jumping' TEXT, 'stamina' TEXT, 'strength' TEXT, 'long_shots' TEXT, 'aggression' TEXT, 'interceptions' TEXT, 'positioning' TEXT, 'vision' TEXT,
                   'penalties' TEXT, 'marking' TEXT, 'standing_tackle' TEXT, 'sliding_tackle' TEXT, 'gk_diving' TEXT, 'gk_handling' TEXT, 'gk_kicking' TEXT, 'gk_positioning' TEXT, 'gk_reflexes' TEXT, 
                FOREIGN KEY (HK_player) REFERENCES HUB_player ("HK"));""")

cursor.execute(f"INSERT INTO new_HSAT_{source_table} SELECT * FROM HSAT_{source_table};")

cursor.execute(f"DROP TABLE HSAT_{source_table};")
cursor.execute(f"ALTER TABLE new_HSAT_{source_table} RENAME TO HSAT_{source_table};")

df_HSAT = pd.read_sql(f'Select * From HSAT_{source_table};', con=db)
df_HSAT.head()

,HK,HK_player,LDTS,EDTS,RS,HD,date,id,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,d97bfcfb02ac4c30963791aae73354e2,36dd02dfaf93f5c8d874cae6cda1e856,2021/02/11 00:15:29,2021/02/11 12:15:29,1,dbf4d28bb8a765a941093977e21f6363,2016-02-18 00:00:00,1,67.0,71.0,right,medium,medium,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,71.0,70.0,45.0,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,525708d4b725b825ae4e56ec2f7471a8,36dd02dfaf93f5c8d874cae6cda1e856,2021/02/11 00:15:29,2021/02/11 12:15:29,1,70f2ce10d8f3e53b2900160809ce001c,2015-11-19 00:00:00,2,67.0,71.0,right,medium,medium,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,71.0,70.0,45.0,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,a17dbb36d53cd7e6ac0ffd474218b1ac,36dd02dfaf93f5c8d874cae6cda1e856,2021/02/11 00:15:29,2021/02/11 12:15:29,1,dcc6fde568a5f2d3dc14d22e7d51bedb,2015-09-21 00:00:00,3,62.0,66.0,right,medium,medium,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,63.0,41.0,45.0,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,111679c8f2baa87961da501441b81656,36dd02dfaf93f5c8d874cae6cda1e856,2021/02/11 00:15:29,2021/02/11 12:15:29,1,b00908236e2703e010df6b0404b2837b,2015-03-20 00:00:00,4,61.0,65.0,right,medium,medium,48.0,43.0,70.0,60.0,43.0,50.0,44.0,38.0,63.0,48.0,60.0,64.0,59.0,46.0,65.0,54.0,58.0,54.0,76.0,34.0,62.0,40.0,44.0,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,a6f5ef166f23e2e434296c609eaf1b81,36dd02dfaf93f5c8d874cae6cda1e856,2021/02/11 00:15:29,2021/02/11 12:15:29,1,e7a1f9b111502f942d3784ddd5725bd8,2007-02-22 00:00:00,5,61.0,65.0,right,medium,medium,48.0,43.0,70.0,60.0,43.0,50.0,44.0,38.0,63.0,48.0,60.0,64.0,59.0,46.0,65.0,54.0,58.0,54.0,76.0,34.0,62.0,40.0,44.0,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


In [629]:
db.commit()

In [630]:
db.close()

In [628]:
get_table_names(db)

['Player_Attributes',
 'Player',
 'Match',
 'League',
 'Country',
 'Team',
 'Team_Attributes',
 'HUB_player',
 'HUB_league',
 'HUB_team',
 'LINK_Match',
 'LINK_Match_Player',
 'HSAT_team',
 'HSAT_league',
 'HSAT_player',
 'HSAT_player_att']

In [609]:
# drops
sql_drop= f"""DROP TABLE IF EXISTS HSAT_player_attributes;"""
cursor.execute(sql_drop)

### LSAT TABELLEN

### LSAT_match

In [633]:
# read source table
df_match = pd.read_sql(f'Select * From Match;', con=db)
df_match.head(7)

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,1.73,3.40,5.00,1.75,3.35,4.20,1.85,3.2,3.5,1.80,3.30,3.75,NaN,NaN,NaN,1.70,3.30,4.33,1.90,3.30,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,1.95,3.20,3.60,1.80,3.30,3.95,1.90,3.2,3.5,1.90,3.20,3.50,NaN,NaN,NaN,1.83,3.30,3.60,1.95,3.30,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,2.38,3.30,2.75,2.40,3.30,2.55,2.60,3.1,2.3,2.50,3.20,2.50,NaN,NaN,NaN,2.50,3.25,2.40,2.63,3.30,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,1.44,3.75,7.50,1.40,4.00,6.80,1.40,3.9,6.0,1.44,3.60,6.50,NaN,NaN,NaN,1.44,3.75,6.00,1.44,4.00,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,5.00,3.50,1.65,5.00,3.50,1.60,4.00,3.3,1.7,4.00,3.40,1.72,NaN,NaN,NaN,4.20,3.40,1.70,4.50,3.50,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67
5,6,1,1,2008/2009,1,2008-09-24 00:00:00,492478,8203,8342,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [638]:
arg_list_match = df_match.columns
arg_list_match

Index(['id', 'country_id', 'league_id', 'season', 'stage', 'date',
       'match_api_id', 'home_team_api_id', 'away_team_api_id',
       'home_team_goal',
       ...
       'SJA', 'VCH', 'VCD', 'VCA', 'GBH', 'GBD', 'GBA', 'BSH', 'BSD', 'BSA'],
      dtype='object', length=115)

In [639]:
arg_list_match = ['season','date', 'home_team_goal', 'away_team_goal', 'goal', 'shoton', 'shotoff', 'foulcommit', 'card', 'cross', 'corner', 'possession']

In [646]:
# read source table
df_match = pd.read_sql(f'Select * From Link_Match;', con=db)
df_match.head(7)

,HK,HK_away_team,HK_home_team,HK_league,LDTS,RS
0,975b30a766e01a9cf33f9358aeb1075d,cf0edb2d39ab5700564c89e812f63db1,35864c52d3d3a5cb853f180500483480,ec308451c1d095c528cfa3c009ea7235,2021-02-10 10:42:24,1
1,b2dadc17e61cd5ae4184e9f214db709b,847d83a8aa5e4694d0f20309d18d4410,e29eebd093f578d532f9999538481eb7,ec308451c1d095c528cfa3c009ea7235,2021-02-10 10:42:24,1
2,a3f9b88cb1376d0cb068e04b12fbfb3b,98422aeb52fc204083f867af3a569758,235b5f87cec01bdc8a19786fe03a9751,ec308451c1d095c528cfa3c009ea7235,2021-02-10 10:42:24,1
3,2a2e8db6d8cbf0f39b05f26650827779,60a1ea9ff920223c747f6e4594daceb7,307b3fd8ae2165be5205d787b3ee287d,ec308451c1d095c528cfa3c009ea7235,2021-02-10 10:42:24,1
4,2974d8e1eb83f76c233313d5d288ab7e,7e8159fb3c3c04e179a855f53e629951,ad4667071684915afb120a03b1cafdd1,ec308451c1d095c528cfa3c009ea7235,2021-02-10 10:42:24,1
5,157e0bdfede3607cabaa5e07182f4d93,e6940aabba6f93dd23cf5a2471450f69,dc7ba61ee159a3d326bc7bf94b27c6f2,ec308451c1d095c528cfa3c009ea7235,2021-02-10 10:42:24,1
6,50e3b703a9ce8fa4d3a035e609403f2e,19e278a98e9bfc72d04122919d135f22,a2031f95829a95f5f2d26c1a6e8aef1e,ec308451c1d095c528cfa3c009ea7235,2021-02-10 10:42:24,1


In [52]:
# generates LSAT_Match
source_table = "Match"
# drops
sql_drop= f"""DROP TABLE IF EXISTS LSAT_{source_table};"""
cursor.execute(sql_drop)

 # creates
sql_create = f"""CREATE TABLE LSAT_{source_table}(HK TEXT NOT NULL PRIMARY KEY, HK_link_match TEXT, LDTS DATETIME, RS INTEGER, HD TEXT, 
                season TEXT, date TEXT, home_team_goal TEXT, away_team_goal TEXT, goal TEXT, shoton TEXT, shotoff TEXT, foulcommit TEXT, card TEXT,
                cross TEXT, corner TEXT, possession TEXT);"""
cursor.execute(sql_create)

# inserts
sql_insert = f"""INSERT INTO LSAT_{source_table}(HK, HK_link_match, LDTS, RS, HD, season, date, home_team_goal, away_team_goal, goal, shoton, shotoff,
                foulcommit, card, cross, corner, possession)
            SELECT 
                season||'_'||away_team_api_id||'_'||home_team_api_id||'_'||league_id||'_'||date||'_'||CAST(1 AS STR)||'_'||datetime('now'),
                season||'_'||away_team_api_id||'_'||home_team_api_id||'_'||league_id||'_'||date||'_'||CAST(1 AS STR),
                datetime('now'), 1,
                season||'_'||date||'_'||home_team_goal||'_'||away_team_goal||'_'||goal||'_'||shoton||'_'||shotoff||'_'||foulcommit||'_'||card||'_'||cross||'_'||
                corner||'_'||possession,
                season, date, home_team_goal, away_team_goal, goal, shoton, shotoff, foulcommit, card, cross,
                corner, possession
            FROM {source_table}"""
cursor.execute(sql_insert)

# converts DF
df = pd.read_sql(f'Select * From LSAT_{source_table};', con=db)

# generate hash
df['HK'] = df['HK'].apply(lambda w: data_vault_hash(w))  # data_vault_hash function
df['HK_link_match'] = df['HK_link_match'].apply(lambda w: data_vault_hash(w))
df['HD'] = df['HD'].apply(lambda w: data_vault_hash(w))

# load df as a table
df.to_sql(f'LSAT_{source_table}', con=db, if_exists='replace',  index=False,
                   dtype={"HK": "TEXT NOT NULL PRIMARY KEY", "LDTS": "DATETIME", "RS": "INTEGER", "HD": 'TEXT', 'season': 'TEXT', 'date': 'TEXT',
                          'home_team_goal': 'TEXT', 'away_team_goal': 'TEXT', 'goal': 'TEXT', 'shoton': 'TEXT', 'shotoff': 'TEXT', 'foulcommit': 'TEXT',
                          'card': 'TEXT', 'cross': 'TEXT', 'corner': 'TEXT', 'possession': 'TEXT'})

cursor.execute(f"""CREATE TABLE new_LSAT_{source_table}(HK TEXT NOT NULL PRIMARY KEY, HK_link_match TEXT, LDTS DATETIME, RS INTEGER, HD TEXT, 
                'season' TEXT, 'date' TEXT, 'home_team_goal' TEXT, 'away_team_goal' TEXT, 'goal' TEXT, 'shoton' TEXT, 'shotoff' TEXT, 'foulcommit' TEXT, 'card' TEXT,
                'cross' TEXT, 'corner' TEXT, 'possession' TEXT, FOREIGN KEY (HK_link_match) REFERENCES LINK_match ("HK"));""")

cursor.execute(f"INSERT INTO new_LSAT_{source_table} SELECT * FROM LSAT_{source_table};")

cursor.execute(f"DROP TABLE LSAT_{source_table};")
cursor.execute(f"ALTER TABLE new_LSAT_{source_table} RENAME TO LSAT_{source_table};")

df_LSAT = pd.read_sql(f'Select * From LSAT_{source_table};', con=db)
df_LSAT.head()

,HK,HK_link_match,LDTS,RS,HD,season,date,home_team_goal,away_team_goal,goal,shoton,shotoff,foulcommit,card,cross,corner,possession
0,e5efec6321f3bad3b28635e9a95e3bee,975b30a766e01a9cf33f9358aeb1075d,2021-02-11 09:05:20,1,None,2008/2009,2008-08-17 00:00:00,1,1,None,None,None,None,None,None,None,None
1,080ccdf33d5489097beadc1094f745a6,b2dadc17e61cd5ae4184e9f214db709b,2021-02-11 09:05:20,1,None,2008/2009,2008-08-16 00:00:00,0,0,None,None,None,None,None,None,None,None
2,c27f78d290af39103567727759498e9d,a3f9b88cb1376d0cb068e04b12fbfb3b,2021-02-11 09:05:20,1,None,2008/2009,2008-08-16 00:00:00,0,3,None,None,None,None,None,None,None,None
3,482700873b0d8be8e30afe45f05b5cb7,2a2e8db6d8cbf0f39b05f26650827779,2021-02-11 09:05:20,1,None,2008/2009,2008-08-17 00:00:00,5,0,None,None,None,None,None,None,None,None
4,f20c17de245f16e54c08a49be5191969,2974d8e1eb83f76c233313d5d288ab7e,2021-02-11 09:05:20,1,None,2008/2009,2008-08-16 00:00:00,1,3,None,None,None,None,None,None,None,None


In [53]:
get_table_names(db)

['Player_Attributes',
 'Player',
 'Match',
 'League',
 'Country',
 'Team',
 'Team_Attributes',
 'HUB_player',
 'HUB_league',
 'HUB_team',
 'LINK_Match',
 'LINK_Match_Player',
 'HSAT_team',
 'HSAT_league',
 'HSAT_player',
 'HSAT_player_att',
 'LSAT_Match']

In [69]:
# read source table
df_match = pd.read_sql(f'Select * From LSAT_Match;', con=db)
df_match.sample(10)

,HK,HK_link_match,LDTS,RS,HD,season,date,home_team_goal,away_team_goal,goal,shoton,shotoff,foulcommit,card,cross,corner,possession
10856,9a107ef79b500d42bd385e7d0ae8a5ed,8d3588215ca086700c170018c7cc7d82,2021-02-11 09:05:20,1,feba95a81147a1faf699090151384414,2009/2010,2009-09-13 00:00:00,0,1,<goal><value><comment>n</comment><stats><goals...,<shoton />,<shotoff />,<foulcommit />,<card><value><comment>y</comment><stats><ycard...,<cross />,<corner />,<possession />
24028,009e47a4264cb5b77b326bd1c09e2381,41ec281f6522c8db2962b135094a67af,2021-02-11 09:05:20,1,edeadbc2fd49029cae7ddfe0d641cf04,2014/2015,2015-04-08 00:00:00,1,1,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><event_incident_typefk>641<...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>51</comment><event...
3699,3d019e4e4f2f625729837eba333ed34f,a86e258f5b990b5afb1b7344b702a705,2021-02-11 09:05:20,1,db3bbe80089bb1a9d14437e0e5b9c07d,2013/2014,2013-12-14 00:00:00,1,0,<goal><value><comment>n</comment><stats><goals...,<shoton><value><event_incident_typefk>152</eve...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>65</comment><event...
7786,2fe5d84a61fd5afbf91bcebfd40eff27,90c7bbd0a6200a36d8bf8936d7c0f8d9,2021-02-11 09:05:20,1,92cfcea90b3bf42c58024f8f57500017,2015/2016,2015-09-23 00:00:00,1,1,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>49</comment><stats...
5934,0384c7a8bfe15de637b77c27a9b6e6c8,d01c0e74bc7e2360213e0c12471f2711,2021-02-11 09:05:20,1,None,2011/2012,2011-10-22 00:00:00,1,0,None,None,None,None,None,None,None,None
17935,de66150834cf8bbcf748fcc98e68091c,628e6886f07d8fb603fc0c82e55124c2,2021-02-11 09:05:20,1,None,2009/2010,2010-01-09 00:00:00,0,1,None,None,None,None,None,None,None,None
23945,4d60b16d5b96180ab7befa38d375459d,a8d829b428807a2d6e51152429e5defe,2021-02-11 09:05:20,1,d3c753dc9e0a046092006a7ae44e5288,2014/2015,2015-02-08 00:00:00,2,5,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>28</comment><event...
17808,7bffe34b536f4279905434132b701726,9d4ca5a0803dcdebd0d0e2c140c651bd,2021-02-11 09:05:20,1,None,2008/2009,2009-05-10 00:00:00,0,0,None,None,None,None,None,None,None,None
22738,01608f503eeb168ebe1966f9b0b0210b,6eae44d132a5487d1c1fb1fe1bc382f3,2021-02-11 09:05:20,1,941bc5c70e98bfe3b50a282c1897f9a0,2011/2012,2011-12-18 00:00:00,0,0,<goal />,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>41</comment><event...
22531,3f41eca735daf6419065623978057fff,b4846f1c9b24a44893c9070554f888c4,2021-02-11 09:05:20,1,ef8f8c6ce1232b2adb142a37ca9a051a,2010/2011,2011-04-17 00:00:00,1,2,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><shoton>1</shoton></s

In [46]:
# read source table
df_match = pd.read_sql(f'Select * From Link_Match;', con=db)
df_match.head()

,HK,HK_away_team,HK_home_team,HK_league,LDTS,RS
0,975b30a766e01a9cf33f9358aeb1075d,cf0edb2d39ab5700564c89e812f63db1,35864c52d3d3a5cb853f180500483480,ec308451c1d095c528cfa3c009ea7235,2021-02-10 10:42:24,1
1,b2dadc17e61cd5ae4184e9f214db709b,847d83a8aa5e4694d0f20309d18d4410,e29eebd093f578d532f9999538481eb7,ec308451c1d095c528cfa3c009ea7235,2021-02-10 10:42:24,1
2,a3f9b88cb1376d0cb068e04b12fbfb3b,98422aeb52fc204083f867af3a569758,235b5f87cec01bdc8a19786fe03a9751,ec308451c1d095c528cfa3c009ea7235,2021-02-10 10:42:24,1
3,2a2e8db6d8cbf0f39b05f26650827779,60a1ea9ff920223c747f6e4594daceb7,307b3fd8ae2165be5205d787b3ee287d,ec308451c1d095c528cfa3c009ea7235,2021-02-10 10:42:24,1
4,2974d8e1eb83f76c233313d5d288ab7e,7e8159fb3c3c04e179a855f53e629951,ad4667071684915afb120a03b1cafdd1,ec308451c1d095c528cfa3c009ea7235,2021-02-10 10:42:24,1


In [65]:
# drops
sql_drop= f"""DROP TABLE IF EXISTS new_LSAT_Wettanbieter;"""
cursor.execute(sql_drop)

### LSAT_Wettanbieter

In [55]:
# read source table
df_match = pd.read_sql(f'Select * From Match;', con=db)
df_match.head(2)

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,4.0,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.2
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3.8,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.6


In [57]:
# arg_list_match = list(df_match.columns)
arg_list_wett = ['B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 'LBH', 'LBD', 'LBA', 'PSH', 'PSD', 'PSA', 'WHH', 'WHD', 'WHA', 'SJH',
                'SJD', 'SJA', 'VCH', 'VCD', 'VCA', 'GBH', 'GBD', 'GBA', 'BSH', 'BSD', 'BSA']

In [76]:
# generates LSAT_Wettanbieter
source_table = "Match"

# drops
sql_drop= f"""DROP TABLE IF EXISTS LSAT_Wettanbieter;"""
cursor.execute(sql_drop)

 # creates
sql_create = f"""CREATE TABLE LSAT_Wettanbieter(HK TEXT NOT NULL PRIMARY KEY, HK_link_match TEXT, LDTS DATETIME, RS INTEGER, HD TEXT, 
                B365H TEXT, B365D TEXT, B365A TEXT, BWH TEXT, BWD TEXT, BWA TEXT, IWH TEXT, IWD TEXT, IWA TEXT, LBH TEXT, LBD TEXT,
                LBA TEXT, PSH TEXT, PSD TEXT, PSA TEXT, WHH TEXT, WHD TEXT, WHA TEXT, SJH TEXT,
                SJD TEXT, SJA TEXT, VCH TEXT, VCD TEXT, VCA TEXT, GBH TEXT, GBD TEXT, GBA TEXT, BSH TEXT, BSD TEXT, BSA TEXT);"""
cursor.execute(sql_create)

# inserts
sql_insert = f"""INSERT INTO LSAT_Wettanbieter(HK, HK_link_match, LDTS, RS, HD, B365H, B365D, B365A, BWH, BWD, BWA, IWH, IWD, IWA, LBH, LBD, LBA, PSH, PSD,
                PSA, WHH, WHD, WHA, SJH, SJD, SJA, VCH, VCD, VCA, GBH, GBD, GBA, BSH, BSD, BSA)
            SELECT 
                season||'_'||away_team_api_id||'_'||home_team_api_id||'_'||league_id||'_'||date||'_'||CAST(1 AS STR)||'_'||datetime('now'),
                season||'_'||away_team_api_id||'_'||home_team_api_id||'_'||league_id||'_'||date||'_'||CAST(1 AS STR),
                datetime('now'), 1,
                season||'_'||date||'_'||CAST(B365H AS STR)||'_'||CAST(B365D AS STR)||'_'||CAST(B365A AS STR)||'_'||CAST(BWH AS STR)||'_'||CAST(BWD AS STR)||'_'||CAST(BWA AS STR)
                ||'_'||CAST(IWH AS STR)||'_'||CAST(IWD AS STR)||'_'||CAST(IWA AS STR)||'_'||CAST(LBH AS STR)||'_'||CAST(LBD AS STR)||'_'||CAST(LBA AS STR)
                ||'_'||CAST(PSH AS STR)||'_'||CAST(PSD AS STR)||'_'||CAST(PSA AS STR)||'_'||CAST(WHH AS STR)||'_'||CAST(WHD AS STR)||'_'||CAST(WHA AS STR)
                ||'_'||CAST(SJH AS STR)||'_'||CAST(SJD AS STR)||'_'||CAST(SJA AS STR)||'_'||CAST(VCH AS STR)||'_'||CAST(VCD AS STR)||'_'||CAST(VCA AS STR)
                ||'_'||CAST(GBH AS STR)||'_'||CAST(GBD AS STR)||'_'||CAST(GBA AS STR)||'_'||CAST(BSH AS STR)||'_'||CAST(BSD AS STR)||'_'||CAST(BSA AS STR),
                B365H, B365D, B365A, BWH, BWD, BWA, IWH, IWD, IWA, LBH, LBD, LBA, PSH, PSD,
                PSA, WHH, WHD, WHA, SJH, SJD, SJA, VCH, VCD, VCA, GBH, GBD, GBA, BSH, BSD, BSA
            FROM {source_table}"""
cursor.execute(sql_insert)

# converts DF
df = pd.read_sql(f'Select * From LSAT_Wettanbieter;', con=db)

# generate hash
df['HK'] = df['HK'].apply(lambda w: data_vault_hash(w))  # data_vault_hash function
df['HK_link_match'] = df['HK_link_match'].apply(lambda w: data_vault_hash(w))
df['HD'] = df['HD'].apply(lambda w: data_vault_hash(w))

# load df as a table
df.to_sql(f'LSAT_Wettanbieter', con=db, if_exists='replace',  index=False,
                   dtype={"HK": "TEXT NOT NULL PRIMARY KEY", "LDTS": "DATETIME", "RS": "INTEGER", "HD": 'TEXT', "B365H": "TEXT", "B365D": "TEXT", "B365A": "TEXT",
                          "BWH": "TEXT", "BWD": "TEXT", "BWA": "TEXT", "IWH": "TEXT", "IWD": "TEXT", "IWA": "TEXT", "LBH": "TEXT", "LBD": "TEXT",
                          "LBA": "TEXT", "PSH": "TEXT", "PSD": "TEXT", "PSA": "TEXT", "WHH": "TEXT", "WHD": "TEXT", "WHA": "TEXT", "SJH": "TEXT",
                          "SJD": "TEXT", "SJA": "TEXT", "VCH": "TEXT", "VCD": "TEXT", "VCA": "TEXT", "GBH": "TEXT", "GBD": "TEXT", "GBA": "TEXT",
                          "BSH": "TEXT", "BSD": "TEXT", "BSA": "TEXT"})

cursor.execute(f"""CREATE TABLE new_LSAT_Wettanbieter(HK TEXT NOT NULL PRIMARY KEY, HK_link_match TEXT, LDTS DATETIME, RS INTEGER, HD TEXT, 
                B365H TEXT, B365D TEXT, B365A TEXT, BWH TEXT, BWD TEXT, BWA TEXT, IWH TEXT, IWD TEXT, IWA TEXT, LBH TEXT, LBD TEXT,
                LBA TEXT, PSH TEXT, PSD TEXT, PSA TEXT, WHH TEXT, WHD TEXT, WHA TEXT, SJH TEXT,
                SJD TEXT, SJA TEXT, VCH TEXT, VCD TEXT, VCA TEXT, GBH TEXT, GBD TEXT, GBA TEXT, BSH TEXT, BSD TEXT, BSA TEXT, FOREIGN KEY (HK_link_match) REFERENCES LINK_match ("HK"));""")

cursor.execute(f"INSERT INTO new_LSAT_Wettanbieter SELECT * FROM LSAT_Wettanbieter;")

cursor.execute(f"DROP TABLE LSAT_Wettanbieter;")
cursor.execute(f"ALTER TABLE new_LSAT_Wettanbieter RENAME TO LSAT_Wettanbieter;")

df_LSAT = pd.read_sql(f'Select * From LSAT_Wettanbieter;', con=db)
df_LSAT.head()

,HK,HK_link_match,LDTS,RS,HD,B365H,B365D,B365A,BWH,BWD,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,cadab90d4b230b81d76132cbc8979a8a,975b30a766e01a9cf33f9358aeb1075d,2021-02-11 09:59:45,1,None,1.73,3.4,5,1.75,3.35,...,4,1.65,3.4,4.5,1.78,3.25,4,1.73,3.4,4.2
1,5b17cb19afe127d7bbcff75c85c87d3d,b2dadc17e61cd5ae4184e9f214db709b,2021-02-11 09:59:45,1,None,1.95,3.2,3.6,1.8,3.3,...,3.8,2,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.6
2,7b77d13a8a7b4242bad0920251371e35,a3f9b88cb1376d0cb068e04b12fbfb3b,2021-02-11 09:59:45,1,None,2.38,3.3,2.75,2.4,3.3,...,2.5,2.35,3.25,2.65,2.5,3.2,2.5,2.3,3.2,2.75
3,804d3e04040b57a614dfb6d8cc5f8250,2a2e8db6d8cbf0f39b05f26650827779,2021-02-11 09:59:45,1,None,1.44,3.75,7.5,1.4,4,...,7.5,1.45,3.75,6.5,1.5,3.75,5.5,1.44,3.75,6.5
4,d6921d9e3d723d31fd4af83ab0cf587d,2974d8e1eb83f76c233313d5d288ab7e,2021-02-11 09:59:45,1,None,5,3.5,1.65,5,3.5,...,1.73,4.5,3.4,1.65,4.5,3.5,1.65,4.75,3.3,1.67


In [77]:
get_table_names(db)

['Player_Attributes',
 'Player',
 'Match',
 'League',
 'Country',
 'Team',
 'Team_Attributes',
 'HUB_player',
 'HUB_league',
 'HUB_team',
 'LINK_Match',
 'LINK_Match_Player',
 'HSAT_team',
 'HSAT_league',
 'HSAT_player',
 'HSAT_player_att',
 'LSAT_Match',
 'LSAT_Wettanbieter']

In [78]:
# read source table
df_match = pd.read_sql(f'Select * From LSAT_Wettanbieter;', con=db)
df_match.sample(10)

,HK,HK_link_match,LDTS,RS,HD,B365H,B365D,B365A,BWH,BWD,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
18764,90792a55eabf61c041cf51c186acdca7,2cf691a65cf3247cb77a510171d82e5b,2021-02-11 09:59:45,1,None,1.2,5.5,13,1.15,6.75,...,11,1.22,6.5,13,1.15,6.75,16,1.14,7,15
23964,b96b548da61bfeefb1fbd28e0fb94587,e3ee14a70d3ea4eff76d242059603b1b,2021-02-11 09:59:45,1,None,1.91,3.5,4.2,1.91,3.5,...,None,1.92,3.6,4.33,None,None,None,None,None,None
25909,5de09bc389b375040bd882c653c17797,cb18c6d8e29464f9ecd1fcedf3654933,2021-02-11 09:59:45,1,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
15584,3ae8505a92032119b6a16909f65bb09e,dec4bcce2e0d20c37400962605007d71,2021-02-11 09:59:45,1,None,3.4,3.3,2.15,3.1,3.5,...,None,3.2,3.4,2.25,None,None,None,None,None,None
6333,74c167c6bc0109697be3284a379eb3ff,f08f33af5445bc22e88ac8d8469bdcd4,2021-02-11 09:59:45,1,fe5638e7907e4404eaf5b50f016abffc,2.63,3,2.88,2.8,3,...,2.8,2.7,3.12,2.88,2.8,3,2.85,2.7,3,2.7
2510,f3b08a268170df2d9534f638e59e7fe8,b590babfa4f7cdfea1db5bbcdb47fe6c,2021-02-11 09:59:45,1,None,3.8,3.4,2,3.75,3.3,...,2.1,3.8,3.4,2.1,3.75,3.2,2,3.75,3.25,2
4778,017b375b510711355bc4bfa68574ab44,539777ef80138d3508db8e3edb4f78a9,2021-02-11 09:59:45,1,None,4.2,3.3,1.91,4.05,3.15,...,2,3.75,3.2,1.91,4,3.1,1.95,4.33,3.1,1.83
15612,1d4b99db2a4c0d0cefd1f0888ee3ef68,374f990525a79bd5d2433d33156ce05f,2021-02-11 09:59:45,1,None,1.25,5.5,12,1.25,5.75,...,None,1.25,6,12,None,None,None,None,None,None
3130,9a3975a157b8895d66cc8cd351c4aca2,4d4e5e8aa299de26f08595474000d1f9,2021-02-11 09:59:45,1,None,1.83,3.5,4.5,1.91,3.25,...,3.9,1.87,3.6,4.5,1.9,3.3,4,2,3.4,3.75
6472,40e0fb300423521978fa5efd28f133a8,8e05bb939d9e21669f9851dc692df47f,2021-02-11 09:59:45,1,c191329ee40542f3d5bc4c2c235b576c,3.5,3.1,2.2,3.5,3.2,...,2.1,3.7,3.2,2.2,3.5,3.2,2.15,3.6,3.2,2.15


In [79]:
db.commit()

In [80]:
db.close()